In [1]:
import sys
import json
from pathlib import Path
import sqlalchemy
import pandas as pd
from utils import * 

In [ ]:
queries_dir = Path('queries/')
enum_query = read_sql(queries_dir / "1.enum_query.sql")
table_query = read_sql(queries_dir / "2.table_query.sql")

datasets_dir = Path("../datasets/")
haccp_dataset_path = datasets_dir / "haccp_dataset_200525221016.csv"
veg_rest_raw_path = datasets_dir / "VegRestrictRawMtrls.json"
allergy_raw_path = datasets_dir / "AllergyRawMtrls.json"

with open(veg_rest_raw_path, encoding='utf-8') as f:
    veg_rest_raw = json.load(f)
    
with open(allergy_raw_path, encoding='utf-8') as f:
    allergy_raw = json.load(f)

In [ ]:
if sys.platform.lower() == 'darwin':
    conn = connect(user='postgres', password='0000', db='projectDB', host='localhost', port=5433)
elif sys.platform == 'win32':
    conn = connect(user='postgres', password='1234', db='projectDB', host='localhost', port=5432)

In [ ]:
initializ_db(conn)

In [5]:
execute_queries(conn, enum_query)
execute_queries(conn, table_query)

In [6]:
haccp_dataset = pd.read_csv(haccp_dataset_path)
if sys.platform.lower() == 'darwin':
    haccp_dataset = haccp_dataset.drop(['rnum', 'productgb', 'prdkindstate'], axis=1)
elif sys.platform == 'win32':
    haccp_dataset = haccp_dataset.drop(['Unnamed: 0', 'rnum', 'productgb', 'prdkindstate'], axis=1)

In [7]:
replace_dict = {
    '(' : ' ',
    ')' : ' ',
    '%' : '%%',
    "'" : '',
    '\n' : ' ',
    '\r' : ' '
}

In [8]:
for data in haccp_dataset.values:
    new_data = []
    for s in data:
        if type(s) == str:
            for org, dst in replace_dict.items():
                s = s.replace(org, dst)
        new_data.append(s)
    reportno, prdnm, rawmtrl, allergy, nutrient, barcode, prdkind, manu, seller, capa, img1, img2 = new_data
    val = "'{reportno}', '{prdnm}', '{rawmtrl}', '{allergy}', '{nutrient}','{barcode}',\
    '{prdkind}', '{manu}', '{seller}', '{capa}', '{img1}', '{img2}'".format(reportno = reportno, prdnm = prdnm, rawmtrl = rawmtrl,
                                                                 allergy = allergy, nutrient = nutrient, barcode = barcode,
                                                                 prdkind = prdkind, manu=manu, seller=seller, capa=capa, img1=img1, img2=img2)
    val = val.replace('(', ' ').replace(')', ' ').replace('%', '%%')
    insert_query = "INSERT INTO foodinfo VALUES({})".format(val)
    execute_query(conn, insert_query, log=False)

In [26]:
# execute_query(conn, "drop table VegRestrictRawMtrls")

# execute_query(conn, """CREATE TABLE VegRestrictRawMtrls
# (
#  veg_kind veg,
#  superRawMtrl TEXT,
#  primary key(veg_kind)
# );""")

In [36]:
for veg, raw_mtrls in veg_rest_raw.items():
    raw_mtrls = '{"' + '", "'.join(raw_mtrls) + '"}'
    insert_query = "INSERT INTO VegRestrictRawMtrls VALUES('{0}', '{1}')".format(veg, raw_mtrls)
    execute_query(conn, insert_query)

In [17]:
# execute_query(conn, "drop table RawMtrls")

# execute_query(conn, """CREATE TABLE RawMtrls
# (
#  rawMtrl TEXT, 
#  superRawMtrl TEXT,
#  isCrossReact BOOLEAN,
#  ParentAllergy TEXT[],
#  probability numeric(2,1),
#  primary key(rawMtrl)
# );""")

In [35]:
for sup_raw, data in allergy_raw.items():
    raws, is_cross, parent_all, prob = data
    raw_mtrls = '{"' + '", "'.join(raws) + '"}'
    for raw in raws:
        if is_cross:
            par_raw_mtrls = '{"' + '", "'.join(parent_all) + '"}'
            val = "'{0}', '{1}', true, '{2}', {3}".format(raw, sup_raw, par_raw_mtrls, prob)
        else:
            val = "'{0}', '{1}', false, null, null".format(raw, sup_raw)
        insert_query = "INSERT INTO RawMtrls VALUES({})".format(val)
        execute_query(conn, insert_query)

In [24]:
parent_all

In [33]:
temp_set = []
for sup_raw, data in allergy_raw.items():
    raws, is_cross, parent_all, prob = data
    if parent_all != None:
        temp_set += parent_all

In [34]:
print(temp_set)

['완두콩', '렌즈콩', '대두', '보리', '호밀', '보리', '호밀', '사과', '자두', '체리', '배', '게', '바닷가재', '햄버거', '염소젖', '말젖', '헤이즐넛', '캐슈넛']
